<a href="https://colab.research.google.com/github/TD008/KNN-model/blob/main/KNN_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and data

In [11]:
import torch
import time
import pickle
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [2]:
# Data import
f = open('data/mnist.pkl', 'rb')
dictionary = pickle.load(f)
f.close()

X_train = dictionary['training_images']
y_train = dictionary['training_labels']
X_test = dictionary['test_images']
y_test = dictionary['test_labels']
#num_labels = 10

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).int()
X_test = torch.from_numpy(X_test).float()

# Getting data to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)



## Distance Forumulas

In [3]:
def calc_L1(x, y):
    diff = x.unsqueeze(1) - y.unsqueeze(0)
    return diff.abs().sum(dim=2)

def calc_L2(x, y):
    diff = x.unsqueeze(1) - y.unsqueeze(0)
    return diff.pow(2).sum(dim=2).sqrt()

def calc_cosine(x, y):
    numerator = (x.unsqueeze(1) * y.unsqueeze(0)).sum(dim=2)

    # Compute L2 norms of each row:
    norm_x = torch.linalg.norm(x, dim=1, keepdim=True)
    norm_y = torch.linalg.norm(y, dim=1, keepdim=True)

    # pairwise product of norms
    denominator = norm_x * norm_y.T

    return numerator / denominator

 ## KNN Function

In [4]:
def knn(x_train, x_test, k, metric, is_cosine=False):
    results = []

    # batches of 50 to not overwhelm the GPU ram
    for i in range(0, x_test.shape[0], 50):
        x_test_batch = x_test[i:i+50]
        batch_distances = metric(x_test_batch, x_train)
        results.append(batch_distances)
        torch.cuda.empty_cache()

    distances = torch.cat(results, dim=0)

    # Find the k smallest distances for each test point
    # We have to find the largest values when using cosine
    values, indices = distances.topk(k, dim=1, largest=is_cosine, sorted = True)

    topk_train = y_train[indices]

    # iterate through the top k values for each point and find the most represented
    # label
    predicted_labels = []
    for neighbor_labels in topk_train:
        majority_label = torch.bincount(neighbor_labels).argmax()
        predicted_labels.append(majority_label)

    predicted_labels = torch.stack(predicted_labels)

    return predicted_labels

## Applying KNN

In [5]:
# Hyperparameters
K_value = 5
distance_metric = calc_L1

start = time.time()
y_hat_L1 = knn(X_train, X_test, k=K_value, metric=distance_metric).cpu().detach().numpy()
execution_time = time.time() - start

In [9]:
# Accuracy
acc = np.mean(y_test == y_hat_L1) * 100
print(f'Ran with K = {K_value} and L1 metric in {execution_time:.2f} seconds \nAccuracy = {acc}')

Ran with K = 5 and L1 metric in 97.65 seconds 
Accuracy = 96.17999999999999


# Testing against linear regression

In [20]:
X_train = X_train.cpu().detach().numpy()
y_train = y_train.cpu().detach().numpy()
X_test = X_test.cpu().detach().numpy()

In [21]:

# Logistic Regression model
start = time.time()
model = LogisticRegression(random_state = 0).fit(X_train, y_train)

# Prediction
y_hat = model.predict(X_test)
execution_time = time.time() - start

# Accuracy
acc = np.mean(y_test == y_hat) * 100
print(f'Accuracy: {acc} \n in {execution_time} seconds')

Accuracy: 92.55 
 in 6.547181606292725 seconds


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
